In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import scipy.io
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
from scipy.io import savemat, loadmat

In [2]:
#data_dir = "../dataset/matfile/"
matfile = scipy.io.loadmat("../allclip1.mat")
xx_all = matfile["xx_all"]
yy_all = matfile["yy_all"]

In [8]:
good_subj_data = np.load("../preprocessed_data/goodsubj_gaze_pos.npz", allow_pickle=True)

In [9]:
gaze_data_goodsubj = good_subj_data["gaze_data_goodsubj"]
gr_goodsubj = good_subj_data["gr_goodsubj"]
group_name = good_subj_data["group_name"]

In [10]:
gaze_data_goodsubj.shape

(104, 3883, 2)

In [11]:
def omitnan(x):
    return x[~np.isnan(x)]

In [ ]:
num_keypoints = 18
sigma = 30
num_subj, num_tracking, _ = gaze_data_goodsubj.shape
print(num_subj, num_tracking, num_keypoints)
gaze_weight = np.zeros((num_subj, num_tracking, num_keypoints)) # to output
for track_idx in tqdm(range(num_tracking)):
    if not np.isnan(xx_all[:, track_idx]).all(): # key point is not nan
        x_key = omitnan(xx_all[:, track_idx])
        y_key = omitnan(yy_all[:, track_idx])
        num_not_nan = len(x_key)
        for subj_idx in range(num_subj):
            x_eye, y_eye = model_eye_pos_all[subj_idx, track_idx]
            d = np.exp(- ((x_key - x_eye)**2 + (y_key - y_eye)**2) / (2*(sigma**2)))
            if np.sum(d) > 1:
                gaze_weight[subj_idx, track_idx, :num_not_nan] = d / np.sum(d) # normalization & save
            elif np.sum(d) <= 1 and np.sum(d) > 0:
                gaze_weight[subj_idx, track_idx, :num_not_nan] = d